In [1]:
#参数设定
import os
import pandas as pd
CODE_MAP = {
    '000852.SH': 'IM',  # 中证1000指数 -> IM期货
    '000905.SH': 'IC',  # 中证500指数 -> IC期货
    '000300.SH': 'IF',  # 沪深300指数 -> IF期货
    '000016.SH': 'IH',  # 上证50指数 -> IH期货
}
PROJECT_PATH = "/Users/yaoyuyang/Desktop/Code/daily_futures_arbitrary_strategy/src"
DATA_PATH = os.path.abspath(os.path.join(PROJECT_PATH, "..",'data'))
RAW_DATA_PATH=os.path.join(DATA_PATH, 'raw')
FILES = {
    '000852.SH':'000852.csv',
    '000905.SH':'000905.csv',
    '000300.SH':'000300.csv',
    '000016.SH':'000016.csv'
}
CAL_TYPE = "udy"
PAIRS = [('IM','IC'), ('IM','IF'), ('IM','IH'),
        ('IC','IF'), ('IC','IH'),
        ('IF','IH')]

In [7]:
#日度调仓
LEVERAGE = 2.0
NV0 = 1.0
frames = []
for code, fname in FILES.items():
    path= os.path.join(RAW_DATA_PATH,fname)
    df = pd.read_csv(path)

    needed = {'time','close'}
    missing = needed - set(df.columns)
    if missing:
        raise ValueError(f"{fname}缺少列：{missing}")

    df = df.copy()
    df["time"] = pd.to_datetime(df["time"])
    df = df.sort_values('time')
    ticker = CODE_MAP[code]
    df = df[['time','close']].rename(columns = {'close':f'p_{ticker}'})
    frames.append(df)

merged = frames[0]
for d in frames[1:]:
    merged = merged.merge(d,on='time',how='inner')
merged['date'] = merged['time'].dt.normalize()

price_cols = [c for c in merged.columns if c.startswith('p_')]
daily_close = merged.groupby('date')[price_cols].last()
prev_close_by_date = daily_close.shift(1)
prev_close_by_date.columns = [c.replace('p_','pc_') for c in prev_close_by_date.columns]

#广播到分钟数据
merged = merged.merge(prev_close_by_date.reset_index(),on='date',how='left')

In [13]:
#计算 OHLC
nv_prev =  {f'{a}{b}':NV0 for (a,b) in PAIRS}
rows = []

for date ,g in merged.groupby('date',sort = True):
    row = {'date': date}
    for long_t,short_t in PAIRS:
        pair = f'{long_t}{short_t}'
        # 当日分钟级价格
        p_long = g[f'p_{long_t}']
        p_short = g[f'p_{short_t}']
        # 当日用于定份额的“昨收”（取第一行即可，已通过 merge 广播）
        pc_long = g[f'pc_{long_t}'].iloc[0]
        pc_short = g[f'pc_{short_t}'].iloc[0]

        nv0_pair = nv_prev[pair]

        #第一日因为没有昨收，所以一直维持前值
        if pd.isna(pc_long) or pd.isna(pc_short):
            o = h = l = c = nv0_pair
        else:
            Rm = (LEVERAGE/2.0)*(p_long/pc_long-p_short/pc_short)
            NVm = nv0_pair*(1+Rm)

            o = float(NVm.iloc[0])
            h = float(NVm.max())
            l = float(NVm.min())
            c = float(NVm.iloc[-1])

            nv_prev[pair] = c
                # 输出列：<PAIR>_<CAL_TYPE>_open/high/low/close
        row[f'{pair}_open']  = o
        row[f'{pair}_high']  = h
        row[f'{pair}_low']   = l
        row[f'{pair}_close'] = c

    rows.append(row)

out = pd.DataFrame(rows).sort_values('date')

In [14]:
out.to_csv(os.path.join(RAW_DATA_PATH,f'index_nv_data.csv'),index=False)